In [ ]:
''' Task: Data pre-processing'''

def preProcessData(eegdata, dgdata):
    '''eegdata.SubjectID=eegdata.SubjectID.astype(int)
    eegdata.VideoID=eegdata.VideoID.astype(int)
    eegdata.predefinedlabel=eegdata.predefinedlabel.astype(int)
    eegdata['user-definedlabeln']=eegdata['user-definedlabeln'].astype(int)

    eegdata.rename(columns = {'predefinedlabel':'TgtLabel', 'user-definedlabeln':'UsrTgtLabel'}, inplace = True)
    demodata.rename(columns={'subject ID':'SubjectID'}, inplace=True)
    #eegdata=eegdata.merge(dgInfo,'inner',left_on='SubjectID', right_on='subject ID')
    
    eegdata=pd.merge(eegdata,demodata,how='inner',on='SubjectID')

    eegdata=pd.get_dummies(eegdata)


    #Rename columns and Create id column as unique row identifier
    eegdata.rename(columns={'SubjectID':'SubjectId'}, inplace=True)
    eegdata.rename(columns={'VideoID':'VideoId'}, inplace=True)
    demodict={' age': 'Age', ' ethnicity_Bengali' : 'EthnicityBengali',
       ' ethnicity_English' : 'EthnicityEnglish', ' ethnicity_Han Chinese' : 'EthnicityHanChinese', 
       ' gender_F' : 'Female', ' gender_M' : 'Male'}
    eegdata.rename(columns=demodict, inplace=True)

    eegdata['SubVdId']=eegdata['SubjectId'].map(str)+'-'+eegdata['VideoId'].map(str)
    eegdata['TimeSecs']=eegdata.groupby(['SubjectId','VideoId']).cumcount()+1
    videoLength=eegdata[['SubjectId', 'VideoId']].value_counts().to_list()
    eegdata=eegdata[eegdata.TimeSecs<=112]'''


    

    eegdata.SubjectID=eegdata.SubjectID.astype(int)
    eegdata.VideoID=eegdata.VideoID.astype(int)
    eegdata.predefinedlabel=eegdata.predefinedlabel.astype(int)
    eegdata['user-definedlabeln']=eegdata['user-definedlabeln'].astype(int)

    eegdata.rename(columns = {'predefinedlabel':'TgtLabel', 'user-definedlabeln':'UsrTgtLabel',
                          'SubjectID':'SubjectId', 'VideoID':'VideoId'}, inplace = True)

    eegdata['SubVdId']=eegdata['SubjectId'].map(str)+'-'+eegdata['VideoId'].map(str)
    eegdata['TimeSecs']=eegdata.groupby(['SubjectId','VideoId']).cumcount()+1
    videoLength=eegdata[['SubjectId', 'VideoId']].value_counts().to_list()
    eegdata=eegdata[eegdata.TimeSecs<=112]


    demodict={'age': 'Age', 'ethnicity_Bengali' : 'EthnicityBengali',
       'ethnicity_English' : 'EthnicityEnglish', 'ethnicity_Han Chinese' : 'EthnicityHanChinese', 
       'gender_F' : 'Female', 'gender_M' : 'Male', 'subject ID':'SubjectId'}
    dgdata.rename(columns=demodict, inplace=True)
    dgdata=pd.get_dummies(dgdata)

    return eegdata, dgdata, videoLength

In [ ]:
'''Task - Create datasets for multiple event epochs - 4,5,6,7,8,9,10,12,15, for any secs segements.
Note - Ensure to pass the appropriate event segment paramater value - epochSize'''

def fftFeatures(signal, topValues):
    fft=np.fft.fft(signal) #Computes the FFT
    magSpect=np.round_(np.abs(fft),2) # Extracts the absolute part from the complex numbers
    
    sortMag=magSpect[np.argsort(magSpect)] #Sorts the magnitude
    uniMag=np.unique(sortMag)[-topValues:] #Extract top unique values 
    
    inter, indMag, indUni=np.intersect1d(magSpect,uniMag, return_indices=True) #Gets the timestep index at which these top values occured
    #eventInd=np.flip(indMag) #np.sort(indMag) #old logic
    eventInd=list(range(len(indMag)))

    for n in range(len(indMag)):
        num=indMag[n]
        if num==0:
            eventInd[n]=0
        else:
            eventInd[n]=np.round(float(1/num),2)
    
    eventInd=np.flip(eventInd)

    return uniMag, eventInd #Returnd the unique top Magnitudes and the respective timestep at which this event magnitude occured

In [ ]:
def genStatFFTFeatures(dataSub, id, epochSize=15):
    #Initialize variables and data structures
    st=0
    end=len(dataSub)

    dat=[]
    lbl=[]
    #Process from the beginning to the end of the trial
    for e in [x for x in range(end) if x%epochSize==0]:
    
        '''Generate statistical features with the time segment attached as part of the new column generated 
        after every epochSize per Subject'''
        dataStat=dataSub[['SubVdId', 'Delta','Theta','Alpha1','Alpha2','Beta1','Beta2','Gamma1','Gamma2']][e:e+epochSize]
        dataStat=dataStat.groupby(['SubVdId']).agg(['mean','std','var','median','min','max', 'skew'])
        dataStat.columns = [''.join(str(i) for i in col) for col in dataStat.columns]
        nameIndex=e+epochSize
        dataStat.columns=[col+'_'+str(nameIndex) for col in dataStat.columns]
        lbl.append(dataStat.columns)
        dat.append(dataStat.values)

    #Create a dataframe by combining the generated column names and statistical features
    result=pd.DataFrame(data=np.concatenate(dat).ravel(), columns=['val'])
    result['rowIndex']=[item for elem in lbl for item in elem]

    magnitude, magnitudeInd=fftFeatures(dataSub['Raw'], topValues=topFFT)
    mag=[]
    magInd=[]

    for m in range(0,len(magnitude)):
        colMag='FFTMag_'+str(m)
        mag.append(colMag)

    for i in range(0,len(magnitudeInd)):
        colMagInd='EventMag_'+str(i)
        magInd.append(colMagInd)
    
    dictKey=[*magnitude, *magnitudeInd]
    dictVal=[*mag, *magInd]
    magDict={'val':dictKey, 'rowIndex':dictVal} #magDict={'val':magnitude, 'colnm':mag}
    fftDf=pd.DataFrame(magDict)
    result=result.append(fftDf)
      
    result=pd.pivot_table(result, columns='rowIndex', values='val')
    return result


In [ ]:
#Input data preparation
def genFeatures(inpdata):
    eegFeatureSet=pd.DataFrame()
    svId=inpdata['SubVdId'].unique()
    magDict={}

    '''Loop through to generate features for all the Subjects.
    Remember to pass the appropriate epochSize here in the function call'''
    for s in svId:
        svDf=inpdata[inpdata['SubVdId']==s]
        eegFeatureSet=eegFeatureSet.append(genStatFFTFeatures(dataSub=svDf, id=s, epochSize=eventMarker))

    '''Create row index for easier identification of the subject combinations.
    The videos presented to subjects are of varying length. 
    Hence for certain shorter video id combinations like 120secs certain features at 140 secs cannot be generated.
    Substitute that with median of the respective feature (not 0) else compute the statistical features till 140+secs'''
    eegFeatureSet.index=inpdata['SubVdId'].unique()


    for n in eegFeatureSet.columns[eegFeatureSet.isnull().any(axis=0)]:
        eegFeatureSet[n].fillna(eegFeatureSet[n].median(),inplace=True)
    
    #eegFeatureSet=eegFeatureSet.replace(np.nan,0)

    #eegFeatureSet=np.log(eegFeatureSet)
    return eegFeatureSet

In [ ]:
def addFeatures(inpdata1, inpdata2, inpdata3):
    inpdata1['SubjectId']=(inpdata1.index.str.slice(0,1)).astype(int)
    inpdata1 = inpdata1.merge(inpdata2, on='SubjectId', how="inner").set_axis(inpdata1.index)
    inpdata1['VideoLen']=inpdata3
    return inpdata1

In [ ]:
def dataScaling(inputdata1, inputdata2, kbestcols):
    '''assign best features from feature engineering. 
    Split this data into train and test in 80:20 ratio.'''

    #kbestcols=['Thetamean_60', 'Alpha1median_60', 'Thetamean_75', 'Thetamax_75']
    inputdata1=inputdata1[kbestcols]

    y=inputdata2[["SubVdId", "UsrTgtLabel"]].groupby("SubVdId").first()
    Xtrain, Xtest, ytrain, ytest=train_test_split(inputdata1, y, test_size=0.2, random_state=randomState)
    
    idx=Xtrain.index.to_list()
    
    #cols=inpdata1.columns

    #Creating the StandardScaler
    scaler = preprocessing.StandardScaler() #scaler=MinMaxScaler()
    output=pd.DataFrame(scaler.fit_transform(Xtrain))

    output.columns=kbestcols
    output.index=idx
    
    #scaler=scaler.fit(inpdata1)
    #output=pd.DataFrame(scaler.transform(inpdata1))
    #output.columns=cols

    #y=inpdata2[["SubVdId", "UsrTgtLabel"]].groupby("SubVdId").first()
    #output["TgtLabel"]=y.UsrTgtLabel.to_list()
    #output.index=inpdata1.index.to_list()

    return output, ytrain, scaler

In [ ]:
#Randomly shuffle the train and test data
def splitTrainTest(input):
    l=list(range(0,100))
    s=input.index.unique()
    random.shuffle(l)
    indTrain, indTest=train_test_split(s[l],test_size=0.2,random_state=0)

    indTrain=list(indTrain)
    indTest=list(indTest)
    return indTrain, indTest

In [ ]:
def featureSelect1(input, noFeatures):
    model=ExtraTreesClassifier()
    model.fit(input.loc[indTrain, input.columns!='TgtLabel'],input.loc[indTrain, input.columns=='TgtLabel'].values.ravel())

    featImportance=pd.Series(model.feature_importances_, index=input.loc[indTrain, input.columns!='TgtLabel'].columns)
    featImportance.nlargest(noFeatures).plot(kind='barh')
    plt.show()
    return featImportance.nlargest(noFeatures)

In [ ]:
def featureSelect2(input, noFeatures):
    bestFeatures=SelectKBest(k=100)

    fit=bestFeatures.fit(input.loc[indTrain, input.columns!='TgtLabel'],input.loc[indTrain, input.columns=='TgtLabel'].values.ravel())
    dfscores=pd.DataFrame(fit.scores_)
    dfcolumns=pd.DataFrame(input.loc[indTrain, input.columns!='TgtLabel'].columns)

    featureScores=pd.concat([dfcolumns, dfscores],axis=1)
    featureScores.columns=['Cols','Scores']

    kbestcols=featureScores.nlargest(noFeatures, 'Scores')['Cols']
    return kbestcols

In [ ]:
def dataVisualize(input):
    pca=PCA(n_components=2)
    pca.fit(input.loc[indTrain, kbestcols]) #pca.fit(eegScaled.loc[:, kbestcols])

    eegPCA=pca.transform(input.loc[indTrain, kbestcols]) #eegPCA=pca.transform(eegScaled.loc[:, kbestcols])

    #eegScaled.shape, eegPCA.shape


    plt.figure(figsize=(8,8))
    plt.scatter(eegPCA[:,0], eegPCA[:,1], c=input.loc[indTrain, input.columns=='TgtLabel'].values.ravel()) #plt.scatter(eegPCA[:,0], eegPCA[:,1], c=y['TgtLabel'])
    plt.xlabel('First Principle Component')
    plt.ylabel('Second Principle Component')


In [ ]:
def plotCV(cvResults,xlab,ylab):
    plt.plot(cvResults['mean_test_score'], label='test')
    plt.plot(cvResults['mean_train_score'], label='train')
    plt.legend(loc='best')
    plt.ylim(ymax=1.0, ymin=0.0)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.show()

In [ ]:
def modelCreate(classifier, param_grid, x, y):
    grid = GridSearchCV(classifier, param_grid, refit=True, verbose=0, scoring='accuracy', return_train_score=True)
    grid.fit(x,y)

    model=grid.best_estimator_
    print(model)

    return model

In [ ]:
def plotConfusionMatrix(predicted, actual):
    
    print(metrics.classification_report(actual, predicted))
    cm= plt.subplot()
    sns.heatmap(confusion_matrix(predicted,actual), annot=True,  ax=cm, cmap='PiYG')  

    # Set the Title, Axis Labels and Class Labels
    cm.set_title('Confusion Matrix for Confusion Prediction')
    cm.set_xlabel('True Labels')
    cm.set_ylabel('Predicted Labels')
    cm.xaxis.set_ticklabels(['Not Confused', 'Confused']) 
    cm.yaxis.set_ticklabels(['Not Confused', 'Confused'])
